# Load Data

In [1]:
c
import math
filename='/Users/wendy/Documents/2017 Fall/CS 534/homework1/spam-dataset/train-features.txt'
train_feature = np.loadtxt(filename)  
filename='/Users/wendy/Documents/2017 Fall/CS 534/homework1/spam-dataset/test-features.txt'
test_feature = np.loadtxt(filename)

filename='/Users/wendy/Documents/2017 Fall/CS 534/homework1/spam-dataset/train-labels.txt'
file=open(filename, 'r')
train_label = file.read().split('\n')
train_label.pop()
filename='/Users/wendy/Documents/2017 Fall/CS 534/homework1/spam-dataset/test-labels.txt'
file=open(filename, 'r')
test_label = file.read().split('\n')
test_label.pop()


''

# Make Dictionary for words and frequency

In [2]:
classvote_0={}
classvote_1={}
for i in range(len(train_feature)):
    votelabel=train_feature[i][1]
    if int(train_label[int(train_feature[i][0])-1])==0:
        if  votelabel in classvote_0:
            classvote_0[votelabel]=classvote_0[votelabel]+train_feature[i][2]
        else:
            classvote_0[votelabel]=train_feature[i][2]
    if int(train_label[int(train_feature[i][0])-1])==1:       
        if  votelabel in classvote_1:
            classvote_1[votelabel]=classvote_1[votelabel]+train_feature[i][2]
        else:
            classvote_1[votelabel]=train_feature[i][2]
    
#print classvote_0

# Calculate total frequency

In [3]:
sum_0=0
sum_1=0
for i in range(len(train_feature)):
    if train_feature[i][0]<=350:
        sum_0+=train_feature[i][2]
    if train_feature[i][0]>350:
        sum_1+=train_feature[i][2]
print sum_0, sum_1

61762.0 91566.0


In [4]:
def getsample(num):
    sample=[]
    for i in range(len(test_feature)-1):
        if test_feature[i][0]==num:
            sample.append(test_feature[i].tolist())
    return sample
        

In [5]:
def predict_for_spam(sample):
    probability=0
    for line in sample:
        if line[1] in classvote_0:
            probability=probability+line[2]*math.log(classvote_0[line[1]]/sum_0)
            #print line[1],classvote_0[line[1]]
        else:
            probability=probability+line[2]*math.log(1/(sum_0+280))
    probability=probability+math.log(0.5)
    return probability
def predict_for_nonspam(sample):
    probability=0
    for line in sample:
        if line[1] in classvote_1:
            probability=probability+line[2]*math.log(classvote_1[line[1]]/sum_1)

        else:
            probability=probability+line[2]*math.log(1/(sum_1+280))
    probability=probability+math.log(0.5)
    return probability
       

# Err_Rate and AUC for Test Data

In [6]:
#####err_rate for test###############
final_result=[]
p0_list=[]
for i in range(260):
    result=[i+1]
    sample=getsample(i+1)
    p0=predict_for_spam(sample)
    p1=predict_for_nonspam(sample)

    if p0>=p1:
        A=p0

        pro_0=p0-(A+math.log(math.exp(p0-A)+math.exp(p1-A)))
        pro_1=p1-(A+math.log(math.exp(p0-A)+math.exp(p1-A)))

        p0_list.append(pro_0)
        result.append(0)
    else:
        A=p1

        pro_0=p0/(A+math.log(math.exp(p0-A)+math.exp(p1-A)))
        pro_1=p1/(A+math.log(math.exp(p0-A)+math.exp(p1-A)))
        p0_list.append(pro_0)

        result.append(1)
        
    if int(test_label[i])==0:
        result.append(0)
    else:
        result.append(1)
    final_result.append(result)

err=0
for i in final_result:
    if i[1]!=i[2]:
        err+=1
print'err_rate:',err

from sklearn.metrics import roc_auc_score
score=p0_list
label=[]
for i in test_label:
    i=int(i)
    label.append(i)
auc = roc_auc_score(label,score)
print 'AUC:',auc

err_rate: 5
AUC: 0.991124260355


The err_rate for the test data is 1.9%; AUC equals to 0.991

In [7]:
def getsample_train(num):
    sample=[]
    for i in range(len(train_feature)-1):
        if train_feature[i][0]==num:
            sample.append(train_feature[i].tolist())
    return sample

# Err_Rate and AUC for Training Data

In [8]:
###########err_rate for train############
p0_list=[]
final_result=[]
for i in range(700):
    result=[i+1]
    sample=getsample_train(i+1)
    p0=predict_for_spam(sample)
    p1=predict_for_nonspam(sample)

    if p0>=p1:
        A=p0

        pro_0=p0-(A+math.log(math.exp(p0-A)+math.exp(p1-A)))
        pro_1=p1-(A+math.log(math.exp(p0-A)+math.exp(p1-A)))

        p0_list.append(pro_0)
        #print p0_list
        result.append(0)
    else:
        A=p1

        pro_0=p0/(A+math.log(math.exp(p0-A)+math.exp(p1-A)))
        pro_1=p1/(A+math.log(math.exp(p0-A)+math.exp(p1-A)))
        p0_list.append(pro_0)

        result.append(1)
        
    if int(train_label[i])==0:
        result.append(0)
    else:
        result.append(1)
    final_result.append(result)
      
err=0
for i in final_result:
    if i[1]!=i[2]:
        err+=1
print'err_rate:',err

from sklearn.metrics import roc_auc_score
score=p0_list
label=[]
for i in train_label:
    i=int(i)
    label.append(i)
auc = roc_auc_score(label,score)
print 'AUC:',auc

err_rate: 9
AUC: 0.977257142857


The err_rate for the test data is 1.28%; AUC equals to 0.977

# compare the results of test and train

Training data has a lower error rate than test data. However, AUC of training data is lower than test data